In [1]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time


In [4]:
def ReadData():
    global a
    global b
    global c
    global query_type 
    global LS
    global nodes_num
    query_type = 0
    LS = []
    input_path = './HepPh.txt'
    id_dic = {}
    input_file = open(input_path,'r')
    nodes_num = 0
    for line in input_file.readlines():
        line = line.replace("\n"," ")
        nodes = line.split()
        id0 = int(nodes[0])
        id1 = int(nodes[1])
        if id0 not in id_dic.keys():
            id_dic[id0] = nodes_num
            nodes_num+=1
        if id1 not in id_dic.keys():
            id_dic[id1] = nodes_num
            nodes_num+=1
    edges = np.zeros([nodes_num,nodes_num])
    input_file_t = open(input_path,'r')
    degree = np.zeros(nodes_num)
    for line in input_file_t.readlines():
        line = line.replace("\n"," ")
        nodes = line.split()
        id0 = id_dic[int(nodes[0])]
        id1 = id_dic[int(nodes[1])]
        edges[id0][id1] = 1
        edges[id1][id0] = 1
        degree[id0]+=1
        
        
    #print(nodes_num)
    #print(degree)
        
    a = []
    b = []
    c = []
    
    if query_type==0:
        matrix_res = np.dot(edges,edges)
        a_dic = {}
        for i in range(nodes_num):
            for j in range(nodes_num):
                if i==j:
                    continue
                a_t = matrix_res[i][j]
                b_t = degree[i]+degree[j]-2*a_t
                
                if a_t not in a_dic.keys():
                    a_dic[a_t] = b_t
                elif b_t>a_dic[a_t]:
                    a_dic[a_t] = b_t
        for a_t in a_dic.keys():
            a.append(a_t)
            b.append(a_dic[a_t])
    
    for i in range(nodes_num):
        degree_t = degree[i]
        c.append(degree_t)
        
    
    
def ComputeLSTriangle():
    global b
    global a
    global LS
    LS = []
    for i in range(2*nodes_num):
        ls_i = 0
        for j in range(len(a)):
            ls_i_t = min(a[j]+ int((i+min(i,b[j]))*1.0/2), nodes_num-2)
            if ls_i_t>ls_i:
                ls_i = ls_i_t
        LS.append(3*ls_i)


def ComputeSSTriangle(beta):
    global SS
    res = 0
    for i in range(nodes_num):
        ls_i = 0
        for j in range(len(a)):
            ls_i_t = min(a[j]+ int((i+min(i,b[j]))*1.0/2), nodes_num-2)
            if ls_i_t>ls_i:
                ls_i = ls_i_t
        ls_i = ls_i*pow(math.e,-1*beta*i)
        #print(ls_i)
        if res<ls_i:
            res = ls_i
            
        SS = res*3
    

def fac(a,k):
    res = 1
    for i in range(k):
        res*=(a-i)
    return res



def ComputeLSTStar(t):
    global c
    res = 0
    global LS
    LS =[]
    max_c = max(c)
    for i in range(2*nodes_num):
        ls_i = min(6*fac(max_c+i,t-1)/fac(t-1,t-1),fac(nodes_num-2,t-1))
        LS.append(ls_i)


def ComputeSSTStar(beta,t):
    global c
    global SS
    res = 0
    max_c = max(c)

    for i in range(nodes_num):
        ls_i = min(6*fac(max_c+i,t-1)/fac(t-1,t-1),fac(nodes_num-2,t-1))*pow(math.e,-1*beta*i)
        #print(ls_i)
        if res<ls_i:
            res = ls_i
    SS = res

In [5]:
def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
    
def F(x):
    return 1/2+1/(4*math.pi)*(math.log(abs(2*x**2+2*math.sqrt(2)*x+2)/abs(abs(2*x**2-2*math.sqrt(2)*x+2)))+2*math.atan(math.sqrt(2)*x+1)+2*math.atan(math.sqrt(2)*x-1))
def inver_F(y):
    #find the solution of F(x)=y
    #Find between -1000000 and 1000000 because F(1000000)=1.0 in python
    if y>1/2:
        low =0.0
        high = 1000000.0
        mid = (high+low)/2
        while abs(high-low)>0.0000001:
            if F(mid)>y:
                high=mid
            elif F(mid)==y:
                return mid
            else: 
                low = mid
            mid = (high+low)/2
            
        return high
    if y==1/2:
        return 0
    
def CauchyNoise():
    a = random.uniform(0.5,1)
    b = inver_F(a)
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
def Smooth(eps, beta, SS, real_Q, delta=0):
    #in our experiments, we only use delat=0 and 1e-6
    if delta==0:
        alpha = eps/20
                
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))

        Q = real_Q+20*SS/eps*CauchyNoise()
        #print(Q)
        indicator=0
        if real_Q<=Q+ 20*noisySS/eps*inver_F(1-beta/4) and real_Q>=Q-20*noisySS/eps*inver_F(1-beta/4):
            indicator=1

        return Q, 20*noisySS/eps*inver_F(1-beta/4),indicator
    else:
        alpha = eps/(4*math.log(1/delta))
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        #print(noisySS) 
        Q = real_Q+4*SS/eps*LapNoise()
        
        
        indicator=0
        if real_Q<=Q+ 4*noisySS/eps*math.log(2/beta) and real_Q>=Q-4*noisySS/eps*math.log(2/beta):
            indicator=1

        return Q, 4*noisySS/eps*math.log(2/beta),indicator

In [7]:
ReadData()


In [265]:
delta=0.0000001
#ComputeSSTriangle(1/(4*math.log(1/delta)))
ComputeSSTStar(0.125/20,3)
print(SS)

721770.0


In [266]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_eps = []
err_eps = []
for i in range(6):
    print("Results for Smooth graph counting, Eps = "+str(eps[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    ComputeSSTStar(eps[i]/20,3)
    for j in range(num_repeat):
        Q,length,indi = Smooth(eps[i], 0.1, SS, 7661801994)
        correct_count+=indi
        errors.append(abs(Q-7661801994))
        lengths.append(length)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[3]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_eps.append(avgLength)
    err_eps.append(errorQuantile)
print(len_eps)
print(err_eps)


Results for Smooth graph counting, Eps = 0.125
correct rate = 0.95
Average CI length = 283064124.0649899
error quantile is 220360923.63250637
CI length/ error quantile = 1.2845477292382974
Results for Smooth graph counting, Eps = 0.25
correct rate = 0.98
Average CI length = 144502930.89859536
error quantile is 88915199.18847752
CI length/ error quantile = 1.6251769350736769
Results for Smooth graph counting, Eps = 0.5
correct rate = 0.94
Average CI length = 70376415.40315332
error quantile is 44928125.31048012
CI length/ error quantile = 1.5664222559212153
Results for Smooth graph counting, Eps = 1
correct rate = 0.98
Average CI length = 35246769.13227285
error quantile is 22381014.852374077
CI length/ error quantile = 1.5748512462353346
Results for Smooth graph counting, Eps = 2
correct rate = 0.96
Average CI length = 17809120.26706101
error quantile is 11303397.674077988
CI length/ error quantile = 1.5755546058423262
Results for Smooth graph counting, Eps = 4
correct rate = 0.99
Aver

In [268]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_beta = []
err_beta = []
for i in range(5):
    print("Results for Smooth graph counting, beta = "+str(beta[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    ComputeSSTStar(eps[3]/20,3)

    for j in range(num_repeat):
        Q,length,indi = Smooth(1, beta[i], SS, 7661801994)
        correct_count+=indi
        errors.append(abs(Q-7661801994))
        lengths.append(length)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[i]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_beta.append(avgLength)
    err_beta.append(errorQuantile)
print(len_beta)
print(err_beta)

Results for Smooth graph counting, beta = 0.01
correct rate = 1.0
Average CI length = 94918315.76989016
error quantile is 50228116.7967453
CI length/ error quantile = 1.8897446654030379
Results for Smooth graph counting, beta = 0.025
correct rate = 1.0
Average CI length = 66074095.52076315
error quantile is 29949255.352175713
CI length/ error quantile = 2.20620161482456
Results for Smooth graph counting, beta = 0.05
correct rate = 0.99
Average CI length = 48674556.63347302
error quantile is 26229612.45833683
CI length/ error quantile = 1.8557100952516086
Results for Smooth graph counting, beta = 0.1
correct rate = 0.99
Average CI length = 34813820.808328316
error quantile is 16727676.541472435
CI length/ error quantile = 2.081210783937353
Results for Smooth graph counting, beta = 0.2
correct rate = 0.94
Average CI length = 25004428.375679992
error quantile is 15083253.076432228
CI length/ error quantile = 1.6577609782832408
[94918315.76989016, 66074095.52076315, 48674556.63347302, 3481

In [24]:
def constructu(Q,eps):
    global u
    global l
    global weight
    global partial_sum
    
    partial_sum = []
    temp = 0
    for i in range(len(LS)):
        temp+=LS[i]
        partial_sum.append(temp)
        
    u = np.zeros(4*nodes_num+1)
    l = np.zeros(4*nodes_num+2)
    u[2*nodes_num] = 0
    l[2*nodes_num] = l[2*nodes_num+1]=Q
    for i in range(2*nodes_num):
        u[2*nodes_num-(i+1)] = u[2*nodes_num+(i+1)] = -(i+1)
        l[2*nodes_num-(i+1)] = Q-partial_sum[i]
        l[2*nodes_num+1+(i+1)] = Q+partial_sum[i]
        
    weight = np.zeros(4*nodes_num+1)
    for i in range(4*nodes_num+1):
        weight[i] = (l[i+1]-l[i])*math.pow(np.e, eps*u[i]/2)
    weight[2*nodes_num] = math.pow(np.e, 0)

    totalWeight = sum(weight)
    weight = weight/totalWeight
    
def constructu_CI(eps, beta):
    global u1
    global weight1
    global u2
    global weight2
    
    N=nodes_num**3
    factor = int(8/eps*np.log(4*N/beta))+3
    print("factor is "+str(factor))
    u1 = np.zeros(4*nodes_num+1)
    u2 = np.zeros(4*nodes_num+1)
    for i in range(4*nodes_num+1):
        if i<=2*nodes_num:
            u1[i] = -abs(u[i]+factor)
            u2[i] = u[i]-factor
        else:
            u1[i] = u[i]-factor
            u2[i] = -abs(u[i]+factor)
            
    weight1 = np.zeros(4*nodes_num+1)
    weight2 = np.zeros(4*nodes_num+1)
    for i in range(4*nodes_num+1):
        weight1[i] = (l[i+1]-l[i])*math.pow(np.e, eps*u1[i]/4)
        weight2[i] = (l[i+1]-l[i])*math.pow(np.e, eps*u2[i]/4)
    weight1[2*nodes_num] = math.pow(np.e, -factor)
    weight2[2*nodes_num] = math.pow(np.e, -factor)
    totalWeight1 = sum(weight1)
    totalWeight2 = sum(weight2)
    #print(totalWeight1,totalWeight2)

    weight1 = weight1/totalWeight1
    weight2 = weight2/totalWeight2

In [25]:
def ladder():

    i = np.random.choice(list(range(4*nodes_num+1)), p=weight)
    if i==int(2*nodes_num):
        x=l[i]
    else:
        x = np.random.randint(l[i], l[i+1],dtype=np.int64)
        

    return x

In [29]:
def LadderCI( Q):
    i1 = np.random.choice(list(range(4*nodes_num+1)), p=weight1)
    i2 = np.random.choice(list(range(4*nodes_num+1)), p=weight2)
    if i1==int(2*nodes_num):
        x1 = l[i1]
    else:
        x1 = np.random.randint(l[i1], l[i1+1],dtype=np.int64)
    if i2==int(2*nodes_num):
        x2 = l[i2]
    else:
        x2 = np.random.randint(l[i2], l[i2+1],dtype=np.int64)
        

    indicator = 0
    if x1<=int(Q)  and x2>=int(Q) :
        indicator=1
        
    return max(x1,0),x2  , indicator
    

In [30]:
#ComputeLSTriangle()
ComputeLSTStar(3)
#Q=20150994
Q=7661801994
constructu(Q,0.125)
constructu_CI(0.125,0.1)
ladder()
LadderCI(Q)

factor is 2042


(0, 23841051008, 1)

In [31]:
eps = [ 0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_eps = []
err_eps = []
for i in range(6):
    print("Results for EM graph counting, Eps = "+str(eps[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    constructu(Q,eps[i])
    constructu_CI(eps[i],beta[3])
    for j in range(num_repeat):
        x1,x2,indi = LadderCI( Q)
        x = ladder()
        correct_count+=indi
        errors.append(abs(x-Q))
        lengths.append((x2-x1)/2)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[3]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_eps.append(avgLength)
    err_eps.append(errorQuantile)
print(len_eps)
print(err_eps)


Results for EM graph counting, Eps = 0.125
factor is 2042
correct rate = 1.0
Average CI length = 11899865400.8
error quantile is 33203429
CI length/ error quantile = 358.3926648298885
Results for EM graph counting, Eps = 0.25
factor is 1022
correct rate = 1.0
Average CI length = 3329005541.52
error quantile is 17242278
CI length/ error quantile = 193.07225770979915
Results for EM graph counting, Eps = 0.5
factor is 512
correct rate = 1.0
Average CI length = 885221915.34
error quantile is 5821110
CI length/ error quantile = 152.07098222503956
Results for EM graph counting, Eps = 1
factor is 257
correct rate = 1.0
Average CI length = 298747204.725
error quantile is 3099236
CI length/ error quantile = 96.39382245334012
Results for EM graph counting, Eps = 2
factor is 130
correct rate = 1.0
Average CI length = 120435401.92
error quantile is 1440560
CI length/ error quantile = 83.60318342866663
Results for EM graph counting, Eps = 4
factor is 66
correct rate = 1.0
Average CI length = 538012

In [21]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_beta = []
err_beta = []
for i in range(5):
    print("Results for EM graph counting, beta = "+str(beta[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    constructu(Q,eps[3])
    constructu_CI(eps[3],beta[i])
    for j in range(num_repeat):
        x1,x2,indi = LadderCI(Q)
        x = ladder()
        correct_count+=indi
        errors.append(abs(x-Q))
        lengths.append((x2-x1)/2)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[i]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_beta.append(avgLength)
    err_beta.append(errorQuantile)
print(len_beta)
print(err_beta)

Results for EM graph counting, beta = 0.01
factor is 276
correct rate = 1.0
Average CI length = 453852.1
error quantile is 17391
CI length/ error quantile = 26.096952446667814
Results for EM graph counting, beta = 0.025
factor is 269
correct rate = 1.0
Average CI length = 441478.385
error quantile is 8788
CI length/ error quantile = 50.23650261720528
Results for EM graph counting, beta = 0.05
factor is 263
correct rate = 1.0
Average CI length = 431166.22
error quantile is 6562
CI length/ error quantile = 65.70652544955806
Results for EM graph counting, beta = 0.1
factor is 257
correct rate = 1.0
Average CI length = 418064.91
error quantile is 5184
CI length/ error quantile = 80.64523726851851
Results for EM graph counting, beta = 0.2
factor is 252
correct rate = 1.0
Average CI length = 408900.18
error quantile is 4997
CI length/ error quantile = 81.82913348008805
[453852.1, 441478.385, 431166.22, 418064.91, 408900.18]
[17391, 8788, 6562, 5184, 4997]
